## Team 10 program Updated v1
### Anthony Ung

#### Modifications Made By AU Before Profiling
- Added the DB API from AU's program
- Changed file path to point inside the Github Repo

- Fixed bug where Team 10 did not account for 2024 being a Leap Year
- Fixed bug where Team 10 restocked all products every day

- Team 10 did not do anything at all with the special product categories or the association rules.
  - AU left this as is and will fix in the v2 Program
- Team 10 limited customers to selecting 5 products.
  - AU left this as is and will fix in the v2 Program

In [2]:
import sqlite3 as lite

'''
    This class provides one common point of interaction with my team's database.
    Everything that writes to the database uses this API.
'''
class db:
    
    def __init__(self, name):
        self.name = rf"{name}"

    def connect(self):
        self.con = lite.connect(self.name)
        self.cur = self.con.cursor()

    def build_table(self, name):      
        self.execute_sql(f'DROP TABLE IF EXISTS {name}')
        self.execute_sql(TABLE_DEFINITIONS[name])
    
    def execute_sql(self, sql, print_results=False):
        if print_results == True:
            results = self.cur.execute(sql).fetchall()
            for row in results:
                print(row)
        else:
            self.cur.execute(sql)

    def execute_sql_values(self, sql, values):
        self.cur.execute(sql, values)

    def commit(self):
        self.con.commit()

    def close(self):
        self.con.commit()
        self.con.close()

In [10]:
import random
import datetime
import csv
from decimal import Decimal

TABLE_DEFINITIONS = {
    'sales_transactions': \
            'CREATE TABLE sales_transactions(' \
                    'date TEXT, ' \
                    'customer_number INT, ' \
                    'sku INT, ' \
                    'salesPrice REAL, ' \
                    'items_left INT, ' \
                    'cases_ordered INT)'
}

# Constants
DAILY_CUSTOMER_MIN, DAILY_CUSTOMER_MAX = 1070, 1100
WEEKEND_EXTRA_CUSTOMERS = 50

Inventory = []

class Product:
    INITIAL_DAYS_SUPPLY = 3
    INITIAL_MILK_SUPPLY = 1.5
    CASE_SIZE = 12
    PRICE_MULTIPLIER = 1.15
    
    def __init__(self, p_name, p_type, sku, price):
        self.p_name = p_name
        self.p_type = p_type
        self.sku = sku
        self.price = price
        self.stock = 0
        self.total_cases_ordered = 0

    def restock(self):
        match self.p_type:
            case 'Milk':
                max_limit = Product.INITIAL_MILK_SUPPLY * 40 # Constant provided by Team 10
            case _:
                max_limit = Product.INITIAL_DAYS_SUPPLY * 40 # Constant provided by Team 10

        num_items_needed = max_limit - self.stock
        num_cases_needed = (num_items_needed + 11) // 12

        self.total_cases_ordered += num_cases_needed
        self.stock += 12*(num_cases_needed)

# Load product data
product_file = r"Products1.txt"

with open('Products1.txt', 'r') as csvfile:

    csv.register_dialect('piper', delimiter='|', quoting=csv.QUOTE_NONE)
    
    for row in csv.DictReader(csvfile, dialect='piper'):
        sku = int(row.get('SKU'))
        product_name = row.get('Product Name')
        product_type = row.get('itemType')
        manufacturer = row.get('Manufacturer')
        base_price = row.get('BasePrice')

        price = float(Decimal(base_price.strip('$')))
        price = round(price * Product.PRICE_MULTIPLIER, 2)

        current_product = Product(\
            p_name = product_name, \
            p_type = product_type, \
            sku = sku, 
            price = price
        )

        match product_type:
            case _:
                Inventory.append(current_product)

db_10 = db('grocery_team_10.db')
db_10.connect()
db_10.build_table('sales_transactions')

start_dt = datetime.datetime(year=2024, month=1, day=1)
end_dt = datetime.datetime(year=2024, month=12, day=31)
current_dt = start_dt
num_days = 1

for product in Inventory:
    product.restock()

while(current_dt <= end_dt):
    daily_customers = random.randint(DAILY_CUSTOMER_MIN, DAILY_CUSTOMER_MAX)
    if current_dt.weekday() >= 5:
       daily_customers += WEEKEND_EXTRA_CUSTOMERS

    if current_dt.weekday() not in (1, 3, 5):
        for product in Inventory:            
            if product.p_type == "Milk":
                product.restock()
    else:
        for product in Inventory:            
            product.restock()

    date_str = current_dt.strftime("%Y%m%d")

    for customer_id in range(daily_customers):
        num_purchases = random.randint(1, 5)
        
        for _ in range(num_purchases):
            product = random.choice(Inventory)
            if product.stock > 0:
            # Item is available, so reduce the inventory by 1
                product.stock -= 1
    
                try:
                    db_10.execute_sql_values('insert into sales_transactions values (?, ?, ?, ?, ?, ?)',
                                            (date_str,customer_id,product.sku,product.price,product.stock,product.total_cases_ordered))
                except Exception as err:
                    print("Error writing to sales_transactions database table", err)

            else:
                # If item is out of stock, substitute with another available item
                substitutes = [p for p in Inventory if p.stock > 0]
                if substitutes:
                    sku = random.choice(substitutes)
                    product.stock -= 1

                    try:
                        db_10.execute_sql_values('insert into sales_transactions values (?, ?, ?, ?, ?, ?)',
                                                (date_str,customer_id,product.sku,product.price,product.stock,product.total_cases_ordered))
                    except Exception as err:
                        print("Error writing to sales_transactions database table", err)
            
    db_10.commit()
    
    num_days += 1
    if(num_days % 30 == 0):
        print(f'{datetime.datetime.now()} - Records committed for {date_str}')

    current_dt += datetime.timedelta(days=1)

db_10.close()
print('Finished')


2025-03-16 20:39:52.353785 - Records committed for 20240129
2025-03-16 20:39:52.932072 - Records committed for 20240228
2025-03-16 20:39:53.536958 - Records committed for 20240329
2025-03-16 20:39:54.114824 - Records committed for 20240428
2025-03-16 20:39:54.737011 - Records committed for 20240528
2025-03-16 20:39:55.233724 - Records committed for 20240627
2025-03-16 20:39:55.678101 - Records committed for 20240727
2025-03-16 20:39:56.105487 - Records committed for 20240826
2025-03-16 20:39:56.740857 - Records committed for 20240925
2025-03-16 20:39:57.167498 - Records committed for 20241025
2025-03-16 20:39:57.571853 - Records committed for 20241124
2025-03-16 20:39:57.975581 - Records committed for 20241224
Finished
